# Model Deployment Class, replicating

In [68]:
from helper_functions import accuracy_fn, download_data, plot_decision_boundary, plot_loss_curves, plot_predictions, set_seeds
# from helper_functions import download_data, set_seeds, plot_loss_curves
from going_modular import engine, data_loaders
from going_modular import predictions
from going_modular.engine import train_step, test_step
from going_modular.utils import save_model
import mlxtend
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
import os
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import random
import requests
import sklearn
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from torchinfo import summary
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import Accuracy, ConfusionMatrix
import torchvision
from torchvision import datasets

from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import Tuple, Dict, List
writer = SummaryWriter()
import zipfile



In [3]:
# see torch and torch vision plus
print (torch.__version__)
print (torchvision.__version__)

2.5.0+cpu
0.20.0+cpu


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def set_seeds (seed: int=42):
    """_summary_

    Args:
        seed (int, optional): _description_. Defaults to 42.
    """ 
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)   

In [54]:
# wet up data paths
# E:\Dropbox\GithubRepo\Udemy\pytorch-deep-learning-main\pytorch-deep-learning-main - Copy\data\pizza_steak_sushi_20_percent
data_20_percent_path = Path('../data/pizza_steak_sushi_20_percent')
train_dir = data_20_percent_path / 'train'
test_dir = data_20_percent_path / 'test'
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2_transforms = effnetb2_weights.transforms()
print (f'effnetb2 transforms: {effnetb2_transforms}')
# effnetb2 = torchvision.models.efficientnet_b2(weights="effnetb2_weights")
effnetb2 = torchvision.models.efficientnet_b2(weights="DEFAULT")

for param in effnetb2.parameters():
    param.requires_grad=False


effnetb2 transforms: ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


In [ ]:
from torchinfo import summary
# effnetb2
summary(model=effnetb2,
        input_size= (1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 1000]            --                   False
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16,

In [24]:
# get input size of last layer
effnetb2_lastlayer = list(effnetb2.children())[-1]
effnetb2_last_child_layer_inpu_count = list(effnetb2_lastlayer.children())[-1].in_features
effnetb2_last_child_layer_inpu_count

1408

In [25]:
set_seeds()
effnetb2.classifier=nn.Sequential(
    nn.Dropout(p = 0.3, inplace=True),
    nn.Linear(in_features=effnetb2_last_child_layer_inpu_count, out_features=3)

)

In [74]:
# function to create effnetb2
def create_effnetb2(output_feature_count:int, drop_out = 0.3, seed:int=42)-> nn.Module:
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms= weights.transforms()
    model = torchvision.models.efficientnet_b2(weights="DEFAULT").to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    last_layer = list(model.children())[-1]
    last_child_layer = list(last_layer.children())[-1]
    last_input = last_child_layer.in_features

    model.classifer = nn.Sequential(
        nn.Dropout(p = drop_out, inplace=True),
        nn.Linear(in_features =last_input, 
                  out_features = output_feature_count).to(device))

    model.name = 'effnetb2'
    print (f'[INFO] created new {model.name} model')
    
    return model, transforms




In [75]:
effnetb2, effnetb2_transforms = create_effnetb2(output_feature_count=3)
effnetb2
print(summary(model=effnetb2,
        input_size= (1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names']))

effnetb2_transforms

[INFO] created new effnetb2 model
Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 1000]            4,227                Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                

ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [76]:
# data loader
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = data_loaders.create_dataloaders(train_dir=train_dir, 
                                                                                          test_dir=test_dir, # use 10% data for testing
                                                                                          transform=effnetb2_transforms, 
                                                                                          batch_size=32)

In [77]:
len (train_dataloader_effnetb2), len (test_dataloader_effnetb2), class_names

(15, 5, ['pizza', 'steak', 'sushi'])

In [78]:
loss_fn = nn.CrossEntropyLoss()
effnetb2_optimizer = torch.optim.Adam(effnetb2.parameters(), lr=1e-3)

In [100]:
# general trainer

def train (model: torch.nn.Module,
           train_dataloader: torch.utils.data.DataLoader,
           test_dataloader: torch.utils.data.DataLoader,
           optimizer: torch.optim.Optimizer,
           loss_fn: torch.nn.Module,
           epochs: int,
           device: torch.device,
           writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:
    
    results = {'train_loss':[],
               'train_acc': [],
               'test_loss': [],
               'test_acc': []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model = model,
                                           dataloader = train_dataloader,
                                           loss_fn = loss_fn,
                                           optimizer=optimizer,
                                           device = device)
        
        test_loss, test_acc = test_step(model = model,
                                        dataloader = test_dataloader,
                                        loss_fn = loss_fn,
                                        device=device)
        
        print (
            f'Epoch: {epoch + 1} |'
            f'train_loss: {train_loss:.4f} |'
            f'train_acc: {train_acc:.4f} |'
            f'test_loss {test_loss:.4f} |'
            f'test_acc:  {test_acc}:.4f'

        )

        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    if writer:
        writer.add_scalars (main_tag = 'Accuracy',
                            tag_scalar_dict = {'train_acc': train_acc,
                                              'test_acc':test_acc},
                            global_step=epoch)
        writer.add_scalars (main_tag = 'Loss',
                           tag_scalar_dict = {'train_loss': train_loss,
                                              'test_loss' : test_loss},
                            global_step = epoch)
        
        writer.add_graph (model = model,
                          input_to_model = torch.randn(32, 3, 224, 224).to(device))# 3 or 1?
    
    writer.close()

    return results

In [101]:
# train effnetb2
set_seeds()

effnetb2_retults = train (model = effnetb2,
                          train_dataloader=train_dataloader_effnetb2,
                          test_dataloader=test_dataloader_effnetb2,
                          optimizer=effnetb2_optimizer,
                          loss_fn = loss_fn,
                          epochs = 10,
                          device=device,
                          writer=writer)

 10%|█         | 1/10 [00:35<05:20, 35.58s/it]

Epoch: 1 |train_loss: 0.1022 |train_acc: 0.9875 |test_loss 0.2902 |test_acc:  0.9255681818181818:.4f


 20%|██        | 2/10 [01:08<04:32, 34.01s/it]

Epoch: 2 |train_loss: 0.0979 |train_acc: 0.9896 |test_loss 0.3408 |test_acc:  0.9130681818181818:.4f


 30%|███       | 3/10 [01:41<03:55, 33.66s/it]

...

Epoch: 8 |train_loss: 0.1035 |train_acc: 0.9792 |test_loss 0.2880 |test_acc:  0.928409090909091:.4f


 90%|█████████ | 9/10 [05:00<00:33, 33.25s/it]

Epoch: 9 |train_loss: 0.1234 |train_acc: 0.9896 |test_loss 0.2518 |test_acc:  0.940909090909091:.4f


100%|██████████| 10/10 [05:33<00:00, 33.34s/it]

Epoch: 10 |train_loss: 0.0793 |train_acc: 0.9917 |test_loss 0.2464 |test_acc:  0.9380681818181819:.4f


In [ ]:
# # test code to see how dtaloader work
# # train_dataloader_effnetb2
# # Create an iterator from the DataLoader
# data_iter = iter(train_dataloader_effnetb2)

# # Get the first batch
# first_batch = next(data_iter)

# # print(first_batch)

# # 
# # Get the first batch
# # batch = next(iter(dataloader))
# input_batch, label_batch = first_batch

# # Check the first element
# first_input = input_batch[0]

# print(f"First input tensor: {first_input}")
# print(f"Requires grad: {first_input.requires_grad}")
# print(f"Grad function: {first_input.grad_fn}")


First input tensor: tensor([[[ 2.0434,  1.9920,  1.9578,  ..., -0.2342, -0.5424, -0.7993],
         [ 2.0263,  1.9407,  1.9064,  ...,  0.1597, -0.0287, -0.2684],
         [ 1.9749,  1.9064,  1.8722,  ...,  0.1426,  0.1254,  0.1083],
         ...,
         [ 2.1119,  2.0777,  2.0777,  ..., -0.2171, -0.3369, -0.3883],
         [ 2.1119,  2.0777,  2.0777,  ..., -0.2171, -0.2342, -0.2684],
         [ 2.0777,  2.1119,  2.1290,  ...,  0.2282, -0.2513, -0.2856]],

        [[ 1.8859,  1.8333,  1.8508,  ..., -1.1604, -1.4405, -1.6681],
         [ 1.8683,  1.7983,  1.7983,  ..., -0.7927, -0.9503, -1.1779],
         [ 1.8508,  1.7983,  1.7633,  ..., -0.8277, -0.8277, -0.8452],
         ...,
         [ 1.6408,  1.6057,  1.6057,  ..., -0.6527, -0.5126, -0.4251],
         [ 1.6583,  1.6232,  1.6057,  ..., -0.7052, -0.4426, -0.3200],
         [ 1.6232,  1.6583,  1.6583,  ..., -0.3025, -0.4951, -0.3725]],

        [[ 1.4722,  1.4722,  1.4897,  ..., -1.5430, -1.6824, -1.8044],
         [ 1.4722,  1.437